In [ ]:
!pip install numba

In [ ]:
import numpy as np
from numba import jit

In [ ]:
#Enter the sum of digits in your enrollment number as seed


def environment(seed):
  np.random.seed(seed)
  p1 = np.random.uniform(0.7,1)
  p2 = np.random.uniform(0,0.6)
  return p1, p2

epsilon = 0.1
r1 = 2
r2 = 0

#Enter the sum of digits in your enrollment number as seed
seed = 43
p1, p2 = environment(15)
print('Value of p1:', p1)
print('Value of p2:', p2)
print('Value of r1:', r1)
print('Value of r2:', r2)
print('Value of epsilon:', epsilon)

Value of p1: 0.9546453091805736
Value of p2: 0.10733755495259908
Value of r1: 2
Value of r2: 0
Value of epsilon: 0.1


Transition function and reward functions

In [ ]:
T =np.zeros([3,3])
T[0]= [p1*(1-epsilon),(1-p1)*(1-epsilon),epsilon]
T[1]= [(1-p2)*(1-epsilon),p2*(1-epsilon),epsilon]
T[2]= [0,0,0]

R = np.zeros(3)
R[0] = r1
R[1] = r2
R[2] = 0

# **Que 1**

---

**states**

*   s1 - 0
*   s2 - 1
*   sT - 3





In [ ]:
def transition(state):
  if state==2:
    return 0.0,-1

  else:
    r =      R[state]
    s_new =  np.random.choice([0,1,2],p = T[state])

    return r, s_new


In [ ]:
transition(0)

(2.0, 0)

In [ ]:
T

array([[0.85918078, 0.04081922, 0.1       ],
       [0.8033962 , 0.0966038 , 0.1       ],
       [0.        , 0.        , 0.        ]])

In [ ]:
@jit#('np.float64(list,int64)',nopython=True)
def g(arr, i):
    tot = 0
    for j in range(0, i):
        tot += arr[j]
    tot = np.sum(arr) - tot
    return tot




<ipython-input-8-35a0fb7bff19>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def g(arr, i):


In [ ]:
@jit
def one(arr,s,i):
  # index = 0
  n = 0
  found = 0
  for z in range(len(arr)):
    if arr[z] ==s:
      n += 1
    if n==i:
      index = z
      found = 1
      break
  return found, index

<ipython-input-9-41dade2e3ac4>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def one(arr,s,i):


Generating episodes

In [ ]:
s_all = []
r_all = []
for i in range(1000):
  episode = []
  r_episode = []
  init = np.random.choice([0,1])
  episode.append(init)
  while(init!=2):
    rew, init = transition(init)
    episode.append(init)
    r_episode.append(rew)
  s_all.append(episode)
  r_all.append(r_episode)


In [ ]:
s_all

In [ ]:
r_all

In [ ]:
one(s_all[0],2,2)

/usr/local/lib/python3.10/dist-packages/numba/core/ir_utils.py:2172: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'arr' of function 'one'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-9-41dade2e3ac4>", line 2:
@jit
def one(arr,s,i):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


(0, 0)

# ***First visit monte carlo estimates***

In [ ]:
# of = []
# indexes =[]
# j = 0
# index = 0
# gf = []
# for i in range(len(s_all)):
#   j , index = one(s_all[i],0,1)
#   if j==1:
#     of.append(j)
#     indexes.append(index)
#     gf.append(g(r_all[i],index))



In [ ]:
@jit
def V(g,o):
  value = sum(g)/sum(o)
  return value

<ipython-input-62-4643071a98f7>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def V(g,o):


In [ ]:
Vfirst = np.zeros(3)
for k in range (3):
  of = []
  indexes =[]
  j = 0
  index = 0
  gf = []
  for i in range(len(s_all)):
    j , index = one(s_all[i],k,1)
    if j==1:
      of.append(j)
      indexes.append(index)
      gf.append(g(r_all[i],index))
  Vfirst[k] = V(gf,of)






In [ ]:
Vfirst

array([18.83262712, 16.95665635,  0.        ])

**Hence first visit monte carlo estimates for the states are**


*   S1 - 18.83262712
*   S2 - 16.95665635
*   ST - 0






# **Second visit monte carlo estimates**

In [ ]:
Vsecond = np.zeros(2)
for k in range (2):
  of = []
  indexes =[]
  j = 0
  index = 0
  gf = []
  for i in range(len(s_all)):
    j , index = one(s_all[i],k,2)
    if j==1:
      of.append(j)
      indexes.append(index)
      gf.append(g(r_all[i],index))
  if of:
    Vsecond[k]  = V(gf,of)

In [ ]:
s_all[1]

[1, 0, 0, 0, 0, 2]

In [ ]:
Vsecond

array([18.80473373, 18.09302326])

**Hence second visit monte carlo estimates for the states are**


*   S1 - 18.80473373
*   S2 - 18.09302326
*   ST - 0

# **every visit monte carlo estimates**

In [ ]:
Vall = np.zeros(2)
for k in range (2):
  of = []
  indexes =[]
  j = 0
  index = 0
  gf = []
  for i in range(len(s_all)):
    for z in range(1,len(s_all[i])+1):

      j , index = one(s_all[i],k,z)
      if j==1:
        of.append(j)
        indexes.append(index)
        gf.append(g(r_all[i],index))
  if of:
    Vall[k]  = V(gf,of)

In [ ]:
Vall

array([19.31375858, 17.274244  ])

**Hence every visit monte carlo estimates for the states are**


*   S1 - 19.31375858
*   S2 - 17.274244
*   ST - 0

# **Last visit monte carlo estimates**

In [ ]:
Vlast = np.zeros(2)
for k in range (2):
  of = []
  indexes =[]
  j = 0
  index = 0
  gf = []
  for i in range(len(s_all)):
    z=0
    for state in s_all[i]:
      if state ==k:
        z = z+1
    j , index = one(s_all[i],k,z)
    if j==1:
      of.append(j)
      indexes.append(index)
      gf.append(g(r_all[i],index))
  if of:
    Vlast[k]  = V(gf,of)

In [ ]:
Vlast

array([ 1.888, 12.718])

**Hence last visit monte carlo estimates for the states are**


*   S1 - 1.888
*   S2 - 12.718
*   ST - 0

# **Values using Bellman's equation**

In [ ]:
va = np.zeros(2)
vb = np.zeros(2)
while(1):

  vb[0] = 2*epsilon +(1-epsilon)*(p1*(2+va[0]) + (1-p1)*(2+va[1]))
  vb[1] = 1*epsilon +(1-epsilon)*(p2*(1+va[1]) + (1-p2)*(1+va[0]))

  if(vb[0]==va[0]):
    if(vb[1]==va[1]):
      break

  va=np.copy(vb)

In [ ]:
vb

array([19.56769164, 18.50861129])